**INTRODUÇÃO**

Nste notebook, acessamos a API do YouTube para coletar informações sobre os vídeos do artista Gilberto Gil, como visualizações, curtidas e comentários ao longo dos anos.

**ÍNDICE**

1. [Consulta à API](#Consulta)

  1.1 [Instalação de bibliotecas e autenticação](#Instalacao)

  1.2 [Estatísticas do canal](#Estatisticas)

  1.3 [Informações de playlists e vídeos](#Playlists)

2. [Leitura e análise dos dados coletados](#Analise)


**EXPEDIENTE**

**Edição DATADOC**

Thays Lavor

**Análise e visualização de dados**

Gabriela Custódio

# 1. Consulta à API <a name="Consulta"></a>

## 1.1 Instalação de bibliotecas e autenticação <a name="Instalacao"></a>

In [ ]:
!pip install --upgrade google-api-python-client

In [ ]:
import pandas as pd
from googleapiclient.discovery import build

In [ ]:
youTubeApiKey = 'api_key'

In [ ]:
youtube = build('youtube', 'v3', developerKey=youTubeApiKey)

## 1.2 Estatísticas do canal <a name="Estatisticas"></a>

In [ ]:
# id do canal do Gilberto Gil

channel_id = 'UCStXqgyucmicmfhpvnJK6cw'

In [ ]:
# informações sobre canal

canalGil = youtube.channels().list(part=['snippet','statistics'], id=channel_id).execute()

In [ ]:
canalGil

{'etag': '9b1MA_DOhRtnisgkqcFIdzQHUDg',
 'items': [{'etag': 'fbe8atDpOF5xpBe2Og1bqklnyrM',
   'id': 'UCStXqgyucmicmfhpvnJK6cw',
   'kind': 'youtube#channel',
   'snippet': {'customUrl': 'gilbertogilmusic',
    'description': 'Canal oficial do cantor e compositor Gilberto Gil no YouTube\n\nSite Oficial: gilbertogil.com.br\nFacebook: facebook.com/gilbertogiloficial\nTwitter: twitter.com/gilbertogil\nInstagram: @gilbertogil',
    'localized': {'description': 'Canal oficial do cantor e compositor Gilberto Gil no YouTube\n\nSite Oficial: gilbertogil.com.br\nFacebook: facebook.com/gilbertogiloficial\nTwitter: twitter.com/gilbertogil\nInstagram: @gilbertogil',
     'title': 'Gilberto Gil'},
    'publishedAt': '2007-12-12T22:24:59Z',
    'thumbnails': {'default': {'height': 88,
      'url': 'https://yt3.ggpht.com/ur0IVtEGbbz6jNAY9cWjFiN1n_14uve70k3gB4XR_SicLEgzwvXhS7eyDQDJgBEGxrakQyMT5Q=s88-c-k-c0x00ffffff-no-nd-rj',
      'width': 88},
     'high': {'height': 800,
      'url': 'https://yt3.gg

In [ ]:
estatisticas_canal = pd.DataFrame.from_dict(canalGil['items'][0]['statistics'], orient='index').reset_index().rename(columns={'index': 'estatísticas', 0: 'valores'})

## 1.3 Informações de playlists e vídeos <a name="Playlists"></a>

In [ ]:
# coletar informações das playlists do canal

nextPageToken = None

playlists_gilberto_gil = []

while True:
  resultado = youtube.playlists().list(part=['snippet'], channelId=channel_id, pageToken=nextPageToken).execute()

  playlists_gilberto_gil += resultado['items']

  nextPageToken = resultado.get('nextPageToken')

  if nextPageToken is None:
    break

# criar lista com os Ids das playlists

ids_playlists = list(map(lambda x: x['id'], playlists_gilberto_gil))
nome_playlists = list(map(lambda x: x['snippet']['title'], playlists_gilberto_gil))

# criar dataframe com informações de id e nome das playlists
info_playlists = dict(zip(ids_playlists, nome_playlists))
df_info_playlists = pd.DataFrame.from_dict(info_playlists, orient='index').reset_index().rename(columns={'index': 'id_playlist', 0:'nome_playlist'})

In [ ]:
df_info_playlists.head()

,id_playlist,nome_playlist
0,PLsYFv2PLHYDyxp25_7KtFtIEtvmw_q6g8,Coleção Gilberto Gil | Google Arts & Culture
1,PLsYFv2PLHYDy6vH_QAMnSWmSemFw_C0kx,Shorts
2,PLsYFv2PLHYDxWdTWpRFQmfFOqcj1n5nDY,DUETOS DO GIL
3,PLsYFv2PLHYDwrvBMCUCgSy2pbY3jILE95,MÊS DAS MULHERES
4,PLsYFv2PLHYDyPnWY7Kcdd8qyQjjTnL57I,MÊS DA CONSCIÊNCIA NEGRA


In [ ]:
# coletar informações dos vídeos das playlists

nextPageToken = None

itens_playlists = []

for playlist in ids_playlists:
  while True:
    videos_playlists = youtube.playlistItems().list(part='snippet', playlistId=playlist, pageToken=nextPageToken).execute()
    itens_playlists += videos_playlists['items']

    nextPageToken = videos_playlists.get('nextPageToken')

    if nextPageToken is None:
      break

# criar lista com Ids de cada vídeo
ids_videos = list(map(lambda x: x['snippet']['resourceId']['videoId'], itens_playlists))
nomes_videos = list(map(lambda x: x['snippet']['title'], itens_playlists))
videos_playlist_id = list(map(lambda x: x['snippet']['playlistId'], itens_playlists))

In [ ]:
# criar dataframe com informações sobre vídeos e playlists

infos_videos_playlist = list(zip(nomes_videos, ids_videos, videos_playlist_id))
df_infos_videos_playlist = pd.DataFrame.from_records(infos_videos_playlist, columns=['titulo_video', 'id_video', 'id_playlist'])
df_infos_videos_playlist_novo = df_infos_videos_playlist.merge(df_info_playlists, on='id_playlist', how='left')

In [ ]:
df_infos_videos_playlist_novo.head()

,titulo_video,id_video,id_playlist,nome_playlist
0,Gilberto Gil no show do álbum Quanta,L0ix9byL4Wc,PLsYFv2PLHYDyxp25_7KtFtIEtvmw_q6g8,Coleção Gilberto Gil | Google Arts & Culture
1,#Shorts | Elza Eterna,5YdDCP7aXXY,PLsYFv2PLHYDy6vH_QAMnSWmSemFw_C0kx,Shorts
2,#Shorts | Gilberto Gil no Desfile da Mangueira...,NPUuX9jJq3c,PLsYFv2PLHYDy6vH_QAMnSWmSemFw_C0kx,Shorts
3,"#Shorts | Jovem, Tire o seu Título de Eleitor!",HrOk_9uyFWI,PLsYFv2PLHYDy6vH_QAMnSWmSemFw_C0kx,Shorts
4,#Shorts | Gilberto Gil no #RockinRio | 04 de S...,zz_zfshPtoY,PLsYFv2PLHYDy6vH_QAMnSWmSemFw_C0kx,Shorts


In [ ]:
# criar lista com dados sobre cada vídeo

lista_dados_video = []

for id_video in ids_videos:
  dados_video_gil = youtube.videos().list(part=['snippet', 'statistics', 'player', 'recordingDetails'], id=id_video).execute()
  lista_dados_video += dados_video_gil['items']

dados_videos_gil = []

for dados_video in lista_dados_video:
  for k, v in dados_video.items():
    titulo_video = dados_video['snippet']['title']
    id_video = dados_video['id']
    data_publicacao = dados_video['snippet']['publishedAt']
    imagem = dados_video['snippet']['thumbnails']['medium']['url']
    for k, v in dados_video['player'].items():
      player_video = dados_video['player']['embedHtml']
    for k, v in dados_video['statistics'].items():
      estatisticas = dados_video['statistics']
      if k == 'commentCount':
        comentarios = dados_video['statistics']['commentCount']
      elif k == 'favoriteCount':
        favoritos = dados_video['statistics']['favoriteCount']
      elif k == 'likeCount':
        likes = dados_video['statistics']['likeCount']
      elif k == 'viewCount':
        visualizacoes = dados_video['statistics']['viewCount']
  dados_videos_gil.append((titulo_video, id_video, data_publicacao, imagem, player_video, estatisticas, comentarios, favoritos, likes, visualizacoes))

In [ ]:
# dataframe com dados dos vídeos do canal de Gilberto Gil

df_videos = pd.DataFrame.from_records(dados_videos_gil, columns=['titulo_video', 'id_video', 'data_publicacao', 'imagem', 'player_video', 'estatisticas', 'comentarios', 'favoritos', 'likes', 'visualizacoes'])
df_videos_novo = df_videos.merge(df_infos_videos_playlist_novo, on=['titulo_video', 'id_video'], how='left')

In [ ]:
# df_videos_novo.to_csv('df_videos_gil_youtube.csv', encoding='utf-8', index=False)

In [ ]:
df_videos_novo.head(2)

,titulo_video,id_video,data_publicacao,imagem,player_video,estatisticas,comentarios,favoritos,likes,visualizacoes,id_playlist,nome_playlist
0,Gilberto Gil no show do álbum Quanta,L0ix9byL4Wc,2021-02-22T16:44:09Z,https://i.ytimg.com/vi/L0ix9byL4Wc/mqdefault.jpg,"<iframe width=""480"" height=""270"" src=""//www.yo...","{'viewCount': '248', 'likeCount': '24', 'favor...",3,0,24,248,PLsYFv2PLHYDyxp25_7KtFtIEtvmw_q6g8,Coleção Gilberto Gil | Google Arts & Culture
1,#Shorts | Elza Eterna,5YdDCP7aXXY,2022-01-26T18:50:14Z,https://i.ytimg.com/vi/5YdDCP7aXXY/mqdefault.jpg,"<iframe width=""480"" height=""360"" src=""//www.yo...","{'viewCount': '3666', 'likeCount': '298', 'fav...",9,0,298,3666,PLsYFv2PLHYDy6vH_QAMnSWmSemFw_C0kx,Shorts


# 2. Leitura e análise dos dados coletados <a name="Analise"></a>

In [ ]:
df_videos_novo['comentarios'] = df_videos_novo['comentarios'].astype(int)
df_videos_novo['likes'] = df_videos_novo['likes'].astype(int)
df_videos_novo['visualizacoes'] = df_videos_novo['visualizacoes'].astype(int)

In [ ]:
df_videos_sem_duplicacao = df_videos_novo.drop_duplicates(subset='id_video')

In [ ]:
df_videos_sem_duplicacao['publicacao'] = df_videos_sem_duplicacao['data_publicacao'].str.slice(start=0, stop=10)
df_videos_sem_duplicacao['publicacao'] = pd.to_datetime(df_videos_sem_duplicacao['publicacao'], format='%Y-%m-%d')
df_videos_sem_duplicacao['ano_publicacao'] = df_videos_sem_duplicacao['publicacao'].dt.year

In [ ]:
df_videos_sem_duplicacao.head(2)

,titulo_video,id_video,data_publicacao,imagem,player_video,estatisticas,comentarios,favoritos,likes,visualizacoes,id_playlist,nome_playlist,publicacao,ano_publicacao
0,Gilberto Gil no show do álbum Quanta,L0ix9byL4Wc,2021-02-22T16:44:09Z,https://i.ytimg.com/vi/L0ix9byL4Wc/mqdefault.jpg,"<iframe width=""480"" height=""270"" src=""//www.yo...","{'viewCount': '248', 'likeCount': '24', 'favor...",3,0,24,248,PLsYFv2PLHYDyxp25_7KtFtIEtvmw_q6g8,Coleção Gilberto Gil | Google Arts & Culture,2021-02-22,2021
1,#Shorts | Elza Eterna,5YdDCP7aXXY,2022-01-26T18:50:14Z,https://i.ytimg.com/vi/5YdDCP7aXXY/mqdefault.jpg,"<iframe width=""480"" height=""360"" src=""//www.yo...","{'viewCount': '3666', 'likeCount': '298', 'fav...",9,0,298,3666,PLsYFv2PLHYDy6vH_QAMnSWmSemFw_C0kx,Shorts,2022-01-26,2022


In [ ]:
# TOP 10 - vídeos com mais likes
top_10_likes = df_videos_sem_duplicacao.sort_values(by='likes', ascending=False)
top_10_likes = top_10_likes.head(10)
df_top_10_likes = top_10_likes[['titulo_video', 'id_video', 'player_video', 'likes']]
df_top_10_likes = df_top_10_likes.rename(columns={'likes':'Likes'})
df_top_10_likes_tidy = pd.melt(df_top_10_likes, id_vars=['titulo_video', 'id_video', 'player_video'], value_vars='likes').rename(columns={'titulo_video': 'Vídeo', 'player_video': 'Assista', 'variable': 'Estatística', 'value': 'Quantidade'})

In [ ]:
# TOP 10 - vídeos com mais visualizações
top_10_visualizacoes = df_videos_sem_duplicacao.sort_values(by='visualizacoes', ascending=False).head(10)
top_10_visualizacoes = top_10_visualizacoes[['titulo_video', 'id_video', 'imagem', 'player_video', 'visualizacoes','publicacao', 'ano_publicacao']]
top_10_visualizacoes = top_10_visualizacoes.rename(columns={'visualizacoes':'Visualizações'})
df_top_10_visualizacoes_tidy = pd.melt(top_10_visualizacoes, id_vars=['titulo_video', 'id_video', 'player_video'], value_vars='Visualizações').rename(columns={'titulo_video': 'Vídeo', 'player_video': 'Assista', 'variable': 'Estatística', 'value': 'Quantidade'})

In [ ]:
# TOP 10 - vídeos com mais comentários
top_10_comentarios = df_videos_sem_duplicacao.sort_values(by='comentarios', ascending=False)
df_top_10_comentarios = top_10_comentarios.head(10)
df_top_10_comentarios = df_top_10_comentarios[['titulo_video', 'id_video', 'imagem', 'player_video', 'comentarios','publicacao', 'ano_publicacao']]
df_top_10_comentarios = df_top_10_comentarios.rename(columns={'comentarios':'Comentários'})
df_top_10_comentarios_tidy = pd.melt(df_top_10_comentarios, id_vars=['titulo_video', 'id_video', 'player_video'], value_vars='Comentários').rename(columns={'titulo_video': 'Vídeo', 'player_video': 'Assista', 'variable': 'Estatística', 'value': 'Quantidade'})

Concatenar os três dataframes de top 10 em formato tidy

In [ ]:
df_top_10_total = pd.concat([df_top_10_likes_tidy, df_top_10_visualizacoes_tidy, df_top_10_comentarios_tidy])
df_top_10_total = df_top_10_total[['Vídeo', 'id_video', 'Assista', 'Estatística', 'Quantidade']]

In [ ]:
df_top_10_total.to_csv('df_top_10_total_gilberto_gil_youtube.csv', index=False)

Estatísticas por ano

In [ ]:
estatisticas_por_ano = df_videos_sem_duplicacao[['ano_publicacao', 'comentarios', 'likes', 'visualizacoes']].groupby(by='ano_publicacao').sum().reset_index().rename(columns={'ano_publicacao': 'Ano de publicação', 'comentarios': 'Comentários', 'likes': 'Likes', 'visualizacoes': 'Visualizações'})
df_estatisticas_por_ano = estatisticas_por_ano.melt(id_vars=['Ano de publicação'], value_vars=['Comentários', 'Likes', 'Visualizações']).rename(columns={'variable': 'Estatística', 'value': 'Quantidade'})
df_estatisticas_por_ano.to_csv('df_estatisticas_por_ano_canal_gilberto_gil_youtube.csv', index=False)